### Backtraking template

#### word-search problem: https://leetcode.com/problems/word-search/solution/

**Algorithm**
There is a certain code pattern for all the algorithms of backtracking. For example, one can find one template in our Explore card of Recursion II.

The skeleton of the algorithm is a loop that iterates through each cell in the grid. For each cell, we invoke the backtracking function (i.e. backtrack()) to check if we would obtain a solution, starting from this very cell.

For the backtracking function backtrack(row, col, suffix), as a DFS algorithm, it is often implemented as a recursive function. The function can be broke down into the following four steps:

Step 1). At the beginning, first we check if we reach the bottom case of the recursion, where the word to be matched is empty, i.e. we have already found the match for each prefix of the word.

Step 2). We then check if the current state is invalid, either the position of the cell is out of the boundary of the board or the letter in the current cell does not match with the first letter of the word.

Step 3). If the current step is valid, we then start the exploration of backtracking with the strategy of DFS. First, we mark the current cell as visited, e.g. any non-alphabetic letter will do. Then we iterate through the four possible directions, namely up, right, down and left. The order of the directions can be altered, to one's preference.

Step 4). At the end of the exploration, we revert the cell back to its original state. Finally we return the result of the exploration.

**Implementation**

In [20]:
class Solution(object):
    def exist(self, board, word):
        """
        :type board: List[List[str]]
        :type word: str
        :rtype: bool
        """
        self.ROWS = len(board)
        self.COLS = len(board[0])
        self.board = board

        for row in range(self.ROWS):
            for col in range(self.COLS):
                if self.backtrack(row, col, word):
                    return True

        # no match found after all exploration
        return False


    def backtrack(self, row, col, suffix):
        # bottom case: we find match for each letter in the word
        if len(suffix) == 0:
            return True

        # Check the current status, before jumping into backtracking
        if row < 0 or row == self.ROWS or col < 0 or col == self.COLS \
                or self.board[row][col] != suffix[0]:
            return False

        ret = False
        # mark the choice before exploring further.
        self.board[row][col] = '#'
        # explore the 4 neighbor directions
        for rowOffset, colOffset in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ret = self.backtrack(row + rowOffset, col + colOffset, suffix[1:])
            # break instead of return directly to do some cleanup afterwards
            if ret: break

        # revert the change, a clean slate and no side-effect
        self.board[row][col] = suffix[0]

        # Tried all directions, and did not find any match
        return ret

### Find repeated sequence

#### repeated-dna-sequences: https://leetcode.com/problems/repeated-dna-sequences/solution/

Follow-up here is to solve the same problem for arbitrary sequence length L, and to check the situation when L is quite large. Hence let's use L = 10 notation everywhere to ease the problem generalisation.

**Rabin-Karp : Constant-time Slice Using Rolling Hash**

Rabin-Karp algorithm is used to perform a multiple pattern search. It's used for plagiarism detection and in bioinformatics to look for similarities in two or more proteins.

The idea is to slice over the string and to compute the hash of the sequence in the sliding window, both in a constant time.

Let's use string  `AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT` as an example. First, convert string to integer array: <br>
    - 'A' -> 0, 'C' -> 1, 'G' -> 2, 'T' -> 3
    
`AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT` -> `00000111110000011111100000222333`. Time to compute hash for the first sequence of length L: `0000011111`. The sequence could be considered as a number in a numeral system with the base 4 and hashed as:

$h_0 = \sum_{i=0}^{L-1} c_i 4^{L-1-i}$

Here $c_{0..4} = 0$ and $c_{5..9} = 1$ are digits of `0000011111`.

Now let's consider the slice `AAAAACCCCC` -> `AAAACCCCCA`. For int arrays that means `0000011111` -> `0000111110`, to remove leading 0 and to add trailing 0. One integer in, and one out, let's recompute the hash:

$h_1 = (h_0 \times 4 - c_0 4 ^L) + c_{L+1}$

Voila, window slice and hash recomputation are both done in a constant time.

**Algorithm** <br>

- Iterate over the start position of sequence : from 1 to N - L.

    - If start == 0, compute the hash of the first sequence `s[0: L]`.

    - Otherwise, compute rolling hash from the previous hash value.

    - If hash is in the hashset, one met a repeated sequence, time to update the output.

    - Otherwise, add hash in the hashset.

- Return output list.

**Implementation**

In [1]:
class Solution:
    def findRepeatedDnaSequences(self, s: str):
        L, n = 10, len(s)
        if n <= L:
            return []
        
        # rolling hash parameters: base a
        a = 4
        aL = pow(a, L) 
        
        # convert string to array of integers
        to_int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
        nums = [to_int.get(s[i]) for i in range(n)]
        
        h = 0
        seen, output = set(), set()
        # iterate over all sequences of length L
        for start in range(n - L + 1):
            # compute hash of the current sequence in O(1) time
            if start != 0:
                h = h * a - nums[start - 1] * aL + nums[start + L - 1]
            # compute hash of the first sequence in O(L) time
            else:
                for i in range(L):
                    h = h * a + nums[i]
            # update output and hashset of seen sequences
            if h in seen:
                output.add(s[start:start + L])
            seen.add(h)
        return output

### Bitwise operation

####  bitwise-and-of-numbers-range: https://leetcode.com/problems/bitwise-and-of-numbers-range/solution/

**Intuition**

Speaking of bit shifting, there is another related algorithm called Brian Kernighan's algorithm which is applied to turn off the rightmost bit of one in a number.

The secret sauce of the Brian Kernighan's algorithm can be summarized as follows:

`When we do AND bit operation between number and number-1, the rightmost bit of one in the original number would be turned off (from one to zero).`

The idea is that for a given range $[m, n]$ (i.e. $m < n$), we could iteratively apply the trick on the number $n$ to turn off its rightmost bit of one until it becomes less or equal than the beginning of the range ($m$), which we denote as $n'$. Finally, we do AND operation between $n'$ and $m$ to obtain the final result.

#### Implementation

In [ ]:
class Solution:
    def rangeBitwiseAnd(self, m: int, n: int) -> int:
        while m < n:
            # turn off rightmost 1-bit
            n = n & (n - 1)
        return m & n

### Priority Queue

#### merge-k-sorted-lists: https://leetcode.com/problems/merge-k-sorted-lists/solution/

<img src="1.png" width="50%" height="50%">
<img src="2.png" width="50%" height="50%">

**Complexity analysis**
- Time complexity: $O(kN)$ where $k$ is the number of linked lists.
     - Almost every selection of node in final linked costs $O(k)$ (${k-1}$ times comparison)
     - There are $N$ nodes in the final linked list.
- Space complexity:
    - $O(n)$ Creating a new linked list costs $O(n)$ space.
    - $O(1)$ It's not hard to apply in-place method - connect selected nodes instead of creating new nodes to fill the new linked list.
    
**Implementation**

In [1]:
from queue import PriorityQueue

class Solution():
    def mergeKLists(self, lists):
        """
        :type lists: List[ListNode]
        :rtype: ListNode
        """
        class Wrapper():
            def __init__(self, node):
                self.node = node
            def __lt__(self, other):
                return self.node.val < other.node.val
        
        head = point = ListNode(0)
        q = PriorityQueue()
        for l in lists:
            if l:
                q.put(Wrapper(l))
        while not q.empty():
            node = q.get().node # NOTE: this is the key point. Here I am getting for free (O(1)) the smallest value.
            point.next = node
            point = point.next
            node = node.next
            if node:
                q.put(Wrapper(node))
        return head.next

### Binary Tree
#### flatten-binary-tree-to-linked-list/: https://leetcode.com/problems/flatten-binary-tree-to-linked-list/solution/

Time complexity: O(N) <br>
Space complexity: (1) <br>

**Intuition**
We'll get to the intuition for this approach in a bit, but first let's talk about the motivation. For any kind of tree traversal, we always have the easiest of solutions which is based on recursion. Next, we have a custom stack based iterative version of the same solution. Finally, we want a tree traversal that doesn't use any kind of additional space at all. There is a well known tree traversal out there that doesn't use any additional space at all. It's known as `Morris Traversal`. Our solution is based off of the same ideology, but Morris Traversal is not a pre-requisite here.

To understand what's difference between the nodes processing of this approach and basic recursion, let's look at a sample tree.

<img src="11.png" width="50%" height="50%">

With recursion, we only re-wire the connections for the "current node" once we are already done processing the left and the right subtrees `completely`. Let's see what that looks like in a figure.

<img src="12.png" width="50%" height="50%">

However, the `postponing` of rewiring of connections on the current node until the left subtree is done, is basically what recursion is. Recursion is all about postponing decisions until something else is completed. In order for us to be able to postpone stuff, we need to use the stack. However, in our current approach we want to get rid of the stack altogether. So, we will have to come up with a `greedy` way that will be costlier in terms of time, but will be space efficient in achieving the same results.

`For a current node, we will check if it has a left child or not. If it does, we will find the last node in the rightmost branch of the subtree rooted at this left child. Once we find this "rightmost" node, we will hook it up with the right child of the current node.`

Let's look at this idea on our current sample tree.

<img src="13.png" width="50%" height="50%">

This might not make a lot of sense just yet. But, bear with me and read on. Let's see what connections we need to establish or shuffle once we find that "rightmost node". We are highlighting "rightmost" here because technically, even without knowing this approach, the node `2`3 would have made much more sense here, right? Instead, we are doing some vodoo with the node `6`. God knows why!

<img src="14.png" width="50%" height="50%">

As mentioned in the previous paragraph, this figure would make much more sense if we had just found out the node `23`, and set its right child to `1` instead of doing all this with `6`. Why did we do that you might ask? Well, it's an optimization of sorts. To find the actual "rightmost" node of subtree, we might have to potentially traverse most of that subtree. Like in our example. To actually get to the node `23`, we would have had to traverse all of the nodes: `2, 6, 44, 23`. Instead, we simply stop at the node `6`. We'll see why that also achieves our final purpose. For now, let's move on.

By doing the following operation for every node, we are simply try ing to move stuff to the right hand side one step at a time. The reason we used the node `6` in the above example and not `23` is the very reason we called this approach somewhat greedy.

Processing of the node `2` is simple since it doesn't have a left child at all. So we have nothing to do here. Let's come over to the node `6` since this is where things get interesting and start to make sense. We'll again use the same logic as before.

`For a current node, we will check if it has a left child or not. If it does, we will find the last node in the rightmost branch of the subtree rooted at this left child. Once we find this "rightmost" node, we will hook it up with the right child of the current node.`

As we can clearly see from the previous figures, the rightmost node here would be `23`. So, let's look at the tree after we are done rewiring the connections.

<img src="15.png" width="50%" height="50%">

Now this looks just like the tree after the recursion would have completed on the left subtree and we rewired the connections, right? Exactly!. The reason we stopped at the `first rightmost node with no right child` was because we would eventually end up `rightyfying` all the subtrees through that connection. Even though before we didn't hook up the node `23`, we were able to do it when we arrived at the node `6` here.

**Algorithm**
- 1) So basically, this is going to be a super short algorithm and a short-er implementation :)
- 2) We use a pointer for traversing the nodes of our tree starting from the root. We have a loop that keeps going until the node pointer becomes null which is when we would be done processing the entire tree.
- 3) For every node we check if it has a left child or not. If it doesn't we simply move on to the right hand side i.e.
    - `node = node.right`
- 4) If the node does have a left child, we find the first node on the rightmost branch of the left subtree which doesn't have a right child i.e. the almost rightmost node.
    - `rightmost = node.left`
    - `while rightmost != null:`
    - `rightmost = rightmost.right`
- 5) Once we find this rightmost node, we rewire the connections as explained in the intuition section.
    - `rightmost.right = node.right`
    - `node.right = node.left`
    - `node.left = null`
    
- 6) And we move on to the right node to continue processing of our tree.

**Implementation**

In [ ]:
class Solution:
    
    def flatten(self, root: TreeNode) -> None:
        """
        Do not return anything, modify root in-place instead.
        """
        
        # Handle the null scenario
        if not root:
            return None
        
        node = root
        while node:
            
            # If the node has a left child
            if node.left:
                
                # Find the rightmost node
                rightmost = node.left
                while rightmost.right:
                    rightmost = rightmost.right
                
                # rewire the connections
                rightmost.right = node.right
                node.right = node.left
                node.left = None
            
            # move on to the right side of the tree
            node = node.right

**Function for printing a tree given the root**

In [133]:
COUNT = [10]
def print2DUtil(root, space) :
 
    # Base case
    if (root == None) :
        return
 
    # Increase distance between levels
    space += COUNT[0]
 
    # Process right child first
    print2DUtil(root.right, space)
 
    # Print current node after space
    # count
    print()
    for i in range(COUNT[0], space):
        print(end = " ")
    print(root.val)
 
    # Process left child
    print2DUtil(root.left, space)
    
    
def print2D(root) :
    # space=[0]
    # Pass initial space count as 0
    print2DUtil(root, 0)

### DFS
#### surrounded-regions: https://leetcode.com/problems/surrounded-regions/solution/

This problem is yet another problem concerning the traversal of 2D grid, e.g. Robot room cleaner.

`As similar to the traversal problems in a tree structure, there are generally two approaches in terms of solution: DFS (Depth-First Search) and BFS (Breadth-First Search).`

One can apply either of the above strategies to traverse the 2D grid, while taking some specific actions to resolve the problems.

Given a traversal strategy (DFS or BFS), there could be a thousand implementations for a thousand people, if we indulge ourselves to exaggerate a bit. However, there are some common neat techniques that we could apply along with both of the strategies, in order to obtain a more optimized solution.

**Approach 1: DFS**

**Intuition**

`The goal of this problem is to mark those captured cells.`

If we are asked to summarize the algorithm in one sentence, it would be that we enumerate all those candidate cells (i.e. the ones filled with `O`), and check one by one if they are captured or not, i.e. we start with a candidate cell (`O`), and then apply either DFS or BFS strategy to explore its surrounding cells.


**Algorithm**

Let us start with the DFS algorithm, which usually results in a more concise code than the BFS algorithm. The algorithm consists of three steps:

- Step 1). We select all the cells that are located on the borders of the board.
- Step 2). Start from each of the above selected cell, we then perform the DFS traversal.
     - If a cell on the border happens to be `O`, then we know that this cell is alive, together with the other `O` cells that are connected to this border cell, based on the description of the problem. Two cells are connected, if there exists a path consisting of only `O` letter that bridges between the two cells.
     - Based on the above conclusion, the goal of our DFS traversal would be to *mark* out all those `connected` `O` cells that is originated from the border, with any distinguished letter such as `E`.
- Step 3). Once we iterate through all border cells, we would then obtain three types of cells:
    - The one with the `X` letter: the cell that we could consider as the wall.
    - The one with the `O` letter: the cells that are spared in our DFS traversal, i.e. these cells has no connection to the border, therefore they are captured. We then should replace these cell with `X` letter.
    
We demonstrate how the DFS works with an example in the following animation (where the white cells that we are marking with a light blue are the cells with `O`).

<img src="21.png" width="50%" height="50%">
<img src="22.png" width="50%" height="50%">
<img src="23.png" width="50%" height="50%">
<img src="24.png" width="50%" height="50%">
<img src="25.png" width="50%" height="50%">
<img src="26.png" width="50%" height="50%">
<img src="27.png" width="50%" height="50%">
<img src="28.png" width="50%" height="50%">

**Implementation**

In [ ]:
class Solution(object):
    def solve(self, board):
        """
        :type board: List[List[str]]
        :rtype: None Do not return anything, modify board in-place instead.
        """
        if not board or not board[0]:
            return

        self.ROWS = len(board)
        self.COLS = len(board[0])

        # Step 1). retrieve all border cells
        from itertools import product
        borders = list(product(range(self.ROWS), [0, self.COLS-1])) \
                + list(product([0, self.ROWS-1], range(self.COLS)))

        # Step 2). mark the "escaped" cells, with any placeholder, e.g. 'E'
        for row, col in borders:
            self.DFS(board, row, col)

        # Step 3). flip the captured cells ('O'->'X') and the escaped one ('E'->'O')
        for r in range(self.ROWS):
            for c in range(self.COLS):
                if board[r][c] == 'O':   board[r][c] = 'X'  # captured
                elif board[r][c] == 'E': board[r][c] = 'O'  # escaped


    def DFS(self, board, row, col):
        if board[row][col] != 'O':
            return
        board[row][col] = 'E'
        if col < self.COLS-1: self.DFS(board, row, col+1)
        if row < self.ROWS-1: self.DFS(board, row+1, col)
        if col > 0: self.DFS(board, row, col-1)
        if row > 0: self.DFS(board, row-1, col)

**Optimizations**

In the above implementation, there are a few techniques that we applied *under the hood*, in order to further optimize our solution. Here we list them one by one.

`Rather than iterating all candidate cells (the ones filled with O), we check only the ones on the borders.`

In the above implementation, our starting points of DFS are those cells that meet two conditions: 
- 1) on the border. 
- 2) filled with `O`.

*As an alternative solution, one might decide to iterate all `O` cells, which is less optimal compared to our starting points.*

As one can see, during DFS traversal, the alternative solution would traverse the cells that eventually might be captured, which is not necessary in our approach.

`Rather than using a sort of visited[cell_index] map to keep track of the visited cells, we simply mark visited cell in place.`

*This technique helps us gain both in the space and time complexity.*

As an alternative approach, one could use a additional data structure to keep track of the visited cells, which goes without saying would require additional memory. And also it requires additional calculation for the comparison. Though one might argue that we could use the hash table data structure for the `visited[]` map, which has the $O(1)$ asymptotic time complexity, but it is still more expensive than the simple comparison on the value of cell.`

`Rather than doing the boundary check within the DFS() function, we do it **before** the invocation of the function.`

As a comparison, here is the implementation where we do the boundary check within the DFS() function.

In [ ]:
def DFS(self, board, row, col):
    if row < 0 or row >= self.ROWS or col < 0 or col >= self.COLS:
        return
    if board[row][col] != 'O':
        return
    board[row][col] = 'E'
    # jump to the neighbors without boundary checks
    for ro, co in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        self.DFS(board, row+ro, col+co)

*This measure reduces the number of recursion, therefore it reduces the overheads with the function calls.*

As trivial as this modification might seem to be, it actually reduces the runtime of the Python implementation from 148 ms to 124 ms, i.e. 16% of reduction, which beats 97% of submissions instead of 77% at the moment.

**Complexity Analysis**

- Time Complexity: $O(N)$ where $N$ is the number of cells in the board. In the worst case where it contains only the `O` cells on the board, we would traverse each cell twice: once during the DFS traversal and the other time during the cell reversion in the last step.
- Space Complexity: $O(N)$ where NN is the number of cells in the board. There are mainly two places that we consume some additional memory.
    - We keep a list of border cells as starting points for our traversal. We could consider the number of border cells is proportional to the total number ($N$) of cells.
    - During the recursive calls of `DFS()` function, we would consume some space in the function call stack, i.e. the call stack will pile up along with the depth of recursive calls. And the maximum depth of recursive calls would be $N$ as in the worst scenario mentioned in the time complexity.
    - As a result, the overall space complexity of the algorithm is $O(N)$.



**Approach 2: BFS**

**Intuition**

`In contrary to the DFS strategy, in BFS (Breadth-First Search) we prioritize the visit of a cell's neighbors before moving further (deeper) into the neighbor's neighbor.`

Though the order of visit might differ between DFS and BFS, eventually both strategies would visit the same set of cells, for most of the 2D grid traversal problems. This is also the case for this problem.

**Algorithm**
We could reuse the bulk of the DFS approach, while simply replacing the `DFS()` function with a `BFS()` function. Here we just elaborate the implementation of the `BFS()` function.
- Essentially we can implement the BFS with the help of queue data structure, which could be of `Array` or more preferably `LinkedList` in Java or `Deque` in Python.
- Through the queue, we maintain the order of visit for the cells. Due to the `FIFO` (First-In First-Out) property of the queue, the one at the head of the queue would have the highest priority to be visited.
- The main logic of the algorithm is a loop that iterates through the above-mentioned queue. At each iteration of the loop, we *pop out* the `head` element from the queue.
     - If the popped element is of the candidate cell (i.e. `O`), we mark it as escaped, otherwise we skip this iteration.
     - For a candidate cell, we then simply append its neighbor cells into the queue, which would get their turns to be visited in the next iterations.
     
As comparison, we demonstrate how BFS works with the same example in DFS, in the following animation.

<img src="31.png" width="50%" height="50%">
<img src="32.png" width="50%" height="50%">
<img src="33.png" width="50%" height="50%">
<img src="34.png" width="50%" height="50%">
<img src="35.png" width="50%" height="50%">
<img src="36.png" width="50%" height="50%">
<img src="37.png" width="50%" height="50%">
<img src="38.png" width="50%" height="50%">
<img src="39.png" width="50%" height="50%">

**Implementation**

In [ ]:
class Solution(object):
    def solve(self, board):
        """
        :type board: List[List[str]]
        :rtype: None Do not return anything, modify board in-place instead.
        """
        if not board or not board[0]:
            return

        self.ROWS = len(board)
        self.COLS = len(board[0])

        # Step 1). retrieve all border cells
        from itertools import product
        borders = list(product(range(self.ROWS), [0, self.COLS-1])) \
                + list(product([0, self.ROWS-1], range(self.COLS)))

        # Step 2). mark the "escaped" cells, with any placeholder, e.g. 'E'
        for row, col in borders:
            #self.DFS(board, row, col)
            self.BFS(board, row, col)

        # Step 3). flip the captured cells ('O'->'X') and the escaped one ('E'->'O')
        for r in range(self.ROWS):
            for c in range(self.COLS):
                if board[r][c] == 'O':   board[r][c] = 'X'  # captured
                elif board[r][c] == 'E': board[r][c] = 'O'  # escaped


    def BFS(self, board, row, col):
        from collections import deque
        queue = deque([(row, col)])
        while queue:
            (row, col) = queue.popleft()
            if board[row][col] != 'O':
                continue
            # mark this cell as escaped
            board[row][col] = 'E'
            # check its neighbor cells
            if col < self.COLS-1: queue.append((row, col+1))
            if row < self.ROWS-1: queue.append((row+1, col))
            if col > 0: queue.append((row, col-1))
            if row > 0: queue.append((row-1, col))

**From BFS to DFS**

In the above implementation of BFS, the fun part is that we could easily convert the BFS strategy to DFS by changing one single line of code. And the obtained DFS implementation is done in iteration, instead of recursion.

`The key is that instead of using the **queue** data structure which follows the principle of FIFO (First-In First-Out), if we use the **stack** data structure which follows the principle of LIFO (Last-In First-Out), we then switch the strategy from BFS to DFS.`

Specifically, at the moment we pop an element from the queue, instead of popping out the head element, we pop the tail element, which then changes the behavior of the container from queue to stack. Here is how it looks like.

In [ ]:
 def DFS(self, board, row, col):
        from collections import deque
        queue = deque([(row, col)])
        while queue:
            # pop out the _tail_ element, rather than the head.
            (row, col) = queue.pop()
            if board[row][col] != 'O':
                continue
            # mark this cell as escaped
            board[row][col] = 'E'
            # check its neighbour cells
            if col < self.COLS-1: queue.append((row, col+1))
            if row < self.ROWS-1: queue.append((row+1, col))
            if col > 0: queue.append((row, col-1))
            if row > 0: queue.append((row-1, col))

Note that, though the above implementations indeed follow the DFS strategy, they are NOT equivalent to the previous `recursive` version of DFS, i.e. they do not produce the exactly same sequence of visit.

In the recursive DFS, we would visit the right-hand side neighbor `(row, col+1)` first, while in the iterative DFS, we would visit the up neighbor `(row-1, col)` first.

In order to obtain the same order of visit as the recursive DFS, one should reverse the processing order of neighbors in the above iterative DFS.

**Complexity**
- Time Complexity: $O(N)$ where $N$ is the number of cells in the board. In the worst case where it contains only the `O` cells on the board, we would traverse each cell twice: once during the BFS traversal and the other time during the cell reversion in the last step.
- Space Complexity: $O(N)$ where $N$ is the number of cells in the board. There are mainly two places that we consume some additional memory.
    - We keep a list of border cells as starting points for our traversal. We could consider the number of border cells is proportional to the total number $(N)$ of cells.
    - Within each invocation of `BFS()` function, we use a queue data structure to hold the cells to be visited. We then need to estimate the upper bound on the size of the queue. *Intuitively we could imagine the unfold of BFS as the structure of an onion*. Each layer of the onion represents the cells that has the same distance to the starting point. Any given moment the queue would contain no more than two layers of *onion*, which in the worst case might cover *almost* all cells in the board.
    - As a result, the overall space complexity of the algorithm is $O(N)$.

